#Callbacks






In [ ]:
import os
from glob import glob
from datetime import datetime
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

from tensorflow.keras import datasets

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard

In [ ]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32,32,3)
num_classes = 10

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

In [ ]:
!unzip /content/gdrive/"My Drive"/fastCampus/cifar.zip

Archive:  /content/gdrive/My Drive/fastCampus/cifar.zip
replace cifar/labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_paths = glob('cifar/train/*.png')[:1000]
test_paths = glob('cifar/test/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [ ]:
def get_class_name(path):
  fname = tf.strings.split(path,'_')[-1]
  lbl_name = tf.strings.regex_replace(fname,'.png','')
  return lbl_name

In [ ]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = tf.unique(train_labels).y.numpy()#y하면결과 부분이 나온다

In [ ]:
def get_label(lbl_name):
  onehot = tf.cast(lbl_name == class_names,tf.uint8)
  return tf.argmax(onehot)

In [ ]:
def read_dataset(path):
  gfile = tf.io.read_file(path)
  image = tf.io.decode_image(gfile)
  #추가로 정규화

  image = tf.cast(image, tf.float32)/255

  class_name = get_class_name(path)
  label = get_label(class_name)



  return image, label

In [ ]:
def image_preprocess(image,label):
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  return image, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(read_dataset)

train_dataset = train_dataset.map(image_preprocess)#data augmentation

train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))
train_dataset = train_dataset.repeat()

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(read_dataset)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.shuffle(buffer_size = len(test_paths))
test_dataset = test_dataset.repeat()

#call backs

In [ ]:
#tensorboard를 저장시켜야한다.
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
logdir

'logs/20200727-145041'

In [ ]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20200727-145041'

In [ ]:
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logdir,
    write_graph = True,
    write_images = True,
    histogram_freq = 1
)

In [ ]:
steps_per_epoch = len(train_paths) // batch_size #batch_size로 나눈걸 1562해야 epcoch
validation_steps = len(test_paths) // batch_size

In [ ]:
model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_per_epoch,
    validation_data = test_dataset,
    validation_steps = validation_steps,
    epochs = num_epochs,
    callbacks=[tensorboard_callback]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
31/31 [==============================] - 2s 60ms/step - loss: 2.2889 - accuracy: 0.1074 - val_loss: 2.2356 - val_accuracy: 0.1126
Epoch 2/10
31/31 [==============================] - 2s 70ms/step - loss: 2.2372 - accuracy: 0.1612 - val_loss: 2.2332 - val_accuracy: 0.1787
Epoch 3/10
31/31 [==============================] - 2s 65ms/step - loss: 2.1436 - accuracy: 0.1973 - val_loss: 2.0715 - val_accuracy: 0.2366
Epoch 4/10
31/31 [==============================] - 2s 66ms/step - loss: 2.0576 - accuracy: 0.2138 - val_loss: 2.0438 - val_accuracy: 0.2118
Epoch 5/10
31/31 [==============================] - 2s 67ms/step - loss: 1.9853 - accuracy: 0.2531 - val_loss: 1.9229 - val_accuracy: 0.2758
Epoch 6/10
31/31 [==============================] - 2s 66ms/step - loss: 1.8982 - accuracy: 0.2696 - val_loss: 1.8793 - val_accuracy: 0.2717
Epoch 7/10
31/31 [==============================] - 2s 68ms/step - loss: 1.848

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 374), started 0:55:15 ago. (Use '!kill 374' to kill it.)

<IPython.core.display.Javascript object>

#lambda callback

In [ ]:
import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [ ]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)#언제 보여줄지 설정가능 epoch이 끝날때 confusion매트릭스를 보여주겠다.

In [ ]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard_callback, cm_callback] #
)

Epoch 1/10
31/31 [==============================] - 2s 79ms/step - loss: 1.7428 - accuracy: 0.3337 - val_loss: 1.7495 - val_accuracy: 0.3430
Epoch 2/10
31/31 [==============================] - 3s 85ms/step - loss: 1.6832 - accuracy: 0.3605 - val_loss: 1.7388 - val_accuracy: 0.3533
Epoch 3/10
31/31 [==============================] - 3s 87ms/step - loss: 1.6546 - accuracy: 0.3698 - val_loss: 1.7273 - val_accuracy: 0.3502
Epoch 4/10
31/31 [==============================] - 3s 92ms/step - loss: 1.6413 - accuracy: 0.3729 - val_loss: 1.6788 - val_accuracy: 0.3760
Epoch 5/10
31/31 [==============================] - 3s 86ms/step - loss: 1.5824 - accuracy: 0.4091 - val_loss: 1.6774 - val_accuracy: 0.3843
Epoch 6/10
31/31 [==============================] - 3s 87ms/step - loss: 1.6043 - accuracy: 0.4042 - val_loss: 1.6581 - val_accuracy: 0.3853
Epoch 7/10
31/31 [==============================] - 3s 87ms/step - loss: 1.5596 - accuracy: 0.4277 - val_loss: 1.7151 - val_accuracy: 0.3605
Epoch 8/10
31

#expert한 버전?

In [ ]:
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)

    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step + (step*epoch))#위에서 나온 이미지를 넣음, 몇 번째 step인가
        tf.summary.scalar('train_loss', train_loss.result(), step=step + (step*epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step=step + (step*epoch))
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

KeyboardInterrupt: ignored

In [ ]:
file_write = tf.summary.create_file_writer(logdir)